# Notebook result

### Training Doc2vec Model in gensim 

Steps: 
1. Load data from mysql metadata database. 

2. Created dataframe for each category 

3. Build namedtuple for doc2vec with prepocessed data

4. Train model on in gensim for each category and save it to disk 


# Future work to be done.  

### Training Doc2vec Model in gensim V2

1. convert pfd to text file and clean text files. 

2. Created dataframe for each category 

3. Build namedtuple for doc2vec with prepocessed text files

4. Train model on in gensim for each category and save it to disk 


### Remarks 

1. This notebook takes time due to the model initalization 

2. Takes upto 2.5 hour on Macbook Air 2017

3. have not handled the edge cases for model saving and traning 

In [46]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [47]:
import os
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import doc2vec
from collections import namedtuple
from gensim.models import doc2vec
import random

In [48]:
# all queries 

astro_ph = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'astro-ph.%';"
cond_mat = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'cond-mat.%';"
esss = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'eess.%';"
econ = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'econ.%';"
cs = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'cs.%';"
hep = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'hep-%';"
maths = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'math.%';"

physics = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'physics.%';"
nlin = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'nlin.%';"
nucl = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'nucl-%';"
q_bio = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'q-bio.%';"
stats = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'stat.%';"
q_fin = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'q-fin.%';"
quant_ph = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'quant-%';"
gr_qc = "select id, arxiv_id, title, abstract, primary_category from arxivOverload.METADATA where primary_category like 'gr-%';"

In [49]:
# Similarity formula
import math

def Cosine(vec1, vec2) :
    result = InnerProduct(vec1,vec2) / (VectorSize(vec1) * VectorSize(vec2))
    return result

def VectorSize(vec) :
    return math.sqrt(sum(math.pow(v,2) for v in vec))

def InnerProduct(vec1, vec2) :
    return sum(v1*v2 for v1,v2 in zip(vec1,vec2))

def Euclidean(vec1, vec2) :
    return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(vec1, vec2)))

def Theta(vec1, vec2) :
    return math.acos(Cosine(vec1,vec2)) + 10

def Triangle(vec1, vec2) :
    theta = math.radians(Theta(vec1,vec2))
    return (VectorSize(vec1) * VectorSize(vec2) * math.sin(theta)) / 2

def Magnitude_Difference(vec1, vec2) :
    return abs(VectorSize(vec1) - VectorSize(vec2))

def Sector(vec1, vec2) :
    ED = Euclidean(vec1, vec2)
    MD = Magnitude_Difference(vec1, vec2)
    theta = Theta(vec1, vec2)
    return math.pi * math.pow((ED+MD),2) * theta/360

def TS_SS(vec1, vec2) :
    return Triangle(vec1, vec2) * Sector(vec1, vec2)

In [50]:
import pandas as pd
import pymysql.cursors

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='password123',
                             db='arxivOverload',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [51]:
astro_ph_df = pd.read_sql(astro_ph, con= connection)

In [52]:
cond_mat_df = pd.read_sql(cond_mat, con=connection)

In [53]:
esss_df = pd.read_sql(esss, con = connection)

In [54]:
econ_df = pd.read_sql(econ, con = connection)

In [55]:
cs_df = pd.read_sql(cs, con = connection)

In [56]:
hep_df = pd.read_sql(hep, con = connection)

In [57]:
maths_df = pd.read_sql(maths, con = connection)

In [58]:
physics_df = pd.read_sql(physics, con = connection)

In [59]:
nlin_df = pd.read_sql(nlin, con = connection)

In [60]:
nucl_df = pd.read_sql(nucl, con = connection)

In [61]:
q_bio_df = pd.read_sql(q_bio, con = connection)

In [62]:
stats_df = pd.read_sql(stats, con = connection)

In [63]:
q_fin_df = pd.read_sql(q_fin, con = connection)

In [64]:
quant_ph_df = pd.read_sql(quant_ph, con = connection)

In [65]:
gr_qc_df = pd.read_sql(gr_qc, con = connection)

In [66]:
dataframes =     [astro_ph_df, cond_mat_df, esss_df, econ_df, cs_df, hep_df, maths_df, physics_df, nlin_df, nucl_df, q_bio_df, stats_df, q_fin_df, quant_ph_df, gr_qc_df]
category_names = ['astro_ph', 'cond_mat', 'esss', 'econ', 'cs', 'hep', 'maths', 'physics', 'nlin', 'nucl', 'q_bio', 'stats', 'q_fin', 'quant_ph', 'gr_qc']

In [67]:
# gensim 
from gensim import corpora
import re
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [71]:
from gensim.models import doc2vec
import random

In [76]:
%%time
def Doc2vec_traning(dataframe_all, category_names_all):
    for dataframe, category_names in zip(dataframe_all, category_names_all): 
        dataframe['abstract'].apply(lambda x: [item for item in x if item not in stop_words])
        text = dataframe['abstract'].tolist()
        tags = dataframe['arxiv_id'].tolist()
        tagged_docs = []
        analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
        for text, tags in zip(dataframe['abstract'].apply(lambda x: re.sub(r'[^a-zA-Z ]', '', str(x))), dataframe['arxiv_id']):
            tags = [tags]
            tagged_docs.append(analyzedDocument(text, tags))
        print("Done Cleaning Text")  
        model = doc2vec.Doc2Vec( size = 1000 # Model initialization
            , window = 400
            , min_count = 1
            , workers = 4)
        model.build_vocab(tagged_docs) # Building vocabulary
        alpha_val = 0.025        # Initial learning rate
        min_alpha_val = 1e-4     # Minimum for linear learning rate decay
        passes = 2              # Number of passes of one document during training
        alpha_delta = (alpha_val - min_alpha_val) / (passes - 1)

        for epoch in range(passes):

            # Shuffling gets better results

            random.shuffle(tagged_docs)

            # Train

            model.alpha, model.min_alpha = alpha_val, alpha_val

            model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)

            # Logs

            print('Completed pass %i at alpha %f' % (epoch + 1, alpha_val))

            # Next run alpha

            alpha_val -= alpha_delta

        model.save(category_names)
        print("Model Saved into Disk")

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.9 µs


In [77]:
%%time
Doc2vec_traning(dataframes, category_names)

/Users/prakritidevverma/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-03-07 11:24:43,162 : INFO : collecting all words and their counts
2018-03-07 11:24:43,164 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-03-07 11:24:43,166 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


Done Cleaning Text


2018-03-07 11:24:44,720 : INFO : PROGRESS: at example #10000, processed 12269109 words (7905658/s), 53 word types, 9993 tags
2018-03-07 11:24:45,812 : INFO : collected 53 word types and 17348 unique tags from a corpus of 17355 examples and 21308796 words
2018-03-07 11:24:45,813 : INFO : Loading a fresh vocabulary
2018-03-07 11:24:45,816 : INFO : min_count=1 retains 53 unique words (100% of original 53, drops 0)
2018-03-07 11:24:45,818 : INFO : min_count=1 leaves 21308796 word corpus (100% of original 21308796, drops 0)
2018-03-07 11:24:45,821 : INFO : deleting the raw counts dictionary of 53 items
2018-03-07 11:24:45,823 : INFO : sample=0.001 downsamples 26 most-common words
2018-03-07 11:24:45,825 : INFO : downsampling leaves estimated 4177677 word corpus (19.6% of prior 21308796)
2018-03-07 11:24:45,827 : INFO : estimated required memory for 53 words and 1000 dimensions: 73312100 bytes
2018-03-07 11:24:45,829 : INFO : resetting layer weights
2018-03-07 11:24:46,515 : INFO : training 

Completed pass 1 at alpha 0.025000


2018-03-07 11:41:32,912 : INFO : EPOCH 1 - PROGRESS: at 0.45% examples, 19587 words/s, in_qsize 7, out_qsize 0
2018-03-07 11:41:33,932 : INFO : EPOCH 1 - PROGRESS: at 0.93% examples, 19901 words/s, in_qsize 7, out_qsize 0
2018-03-07 11:41:34,939 : INFO : EPOCH 1 - PROGRESS: at 1.48% examples, 20805 words/s, in_qsize 7, out_qsize 0
2018-03-07 11:41:35,947 : INFO : EPOCH 1 - PROGRESS: at 2.02% examples, 21459 words/s, in_qsize 7, out_qsize 0
2018-03-07 11:41:36,958 : INFO : EPOCH 1 - PROGRESS: at 2.52% examples, 21266 words/s, in_qsize 8, out_qsize 0
2018-03-07 11:41:38,112 : INFO : EPOCH 1 - PROGRESS: at 2.94% examples, 20272 words/s, in_qsize 8, out_qsize 0
2018-03-07 11:41:39,153 : INFO : EPOCH 1 - PROGRESS: at 3.49% examples, 20425 words/s, in_qsize 7, out_qsize 0
2018-03-07 11:41:40,217 : INFO : EPOCH 1 - PROGRESS: at 4.05% examples, 20631 words/s, in_qsize 8, out_qsize 0
2018-03-07 11:41:41,341 : INFO : EPOCH 1 - PROGRESS: at 4.64% examples, 20677 words/s, in_qsize 8, out_qsize 0
2

...

Done Cleaning Text


2018-03-07 13:54:47,033 : INFO : collected 53 word types and 1305 unique tags from a corpus of 1307 examples and 1257583 words
2018-03-07 13:54:47,035 : INFO : Loading a fresh vocabulary
2018-03-07 13:54:47,038 : INFO : min_count=1 retains 53 unique words (100% of original 53, drops 0)
2018-03-07 13:54:47,039 : INFO : min_count=1 leaves 1257583 word corpus (100% of original 1257583, drops 0)
2018-03-07 13:54:47,041 : INFO : deleting the raw counts dictionary of 53 items
2018-03-07 13:54:47,043 : INFO : sample=0.001 downsamples 24 most-common words
2018-03-07 13:54:47,045 : INFO : downsampling leaves estimated 234025 word corpus (18.6% of prior 1257583)
2018-03-07 13:54:47,047 : INFO : estimated required memory for 53 words and 1000 dimensions: 5931500 bytes
2018-03-07 13:54:47,049 : INFO : resetting layer weights
2018-03-07 13:54:47,125 : INFO : training model with 4 workers on 53 vocabulary and 1000 features, using sg=0 hs=0 sample=0.001 negative=5 window=400
2018-03-07 13:54:48,207 :

Completed pass 1 at alpha 0.025000


2018-03-07 13:55:32,087 : INFO : EPOCH 1 - PROGRESS: at 9.64% examples, 22978 words/s, in_qsize 8, out_qsize 0
2018-03-07 13:55:33,088 : INFO : EPOCH 1 - PROGRESS: at 22.88% examples, 26397 words/s, in_qsize 8, out_qsize 0
2018-03-07 13:55:34,131 : INFO : EPOCH 1 - PROGRESS: at 34.05% examples, 26040 words/s, in_qsize 8, out_qsize 0
2018-03-07 13:55:35,191 : INFO : EPOCH 1 - PROGRESS: at 46.60% examples, 26673 words/s, in_qsize 7, out_qsize 0
2018-03-07 13:55:36,236 : INFO : EPOCH 1 - PROGRESS: at 57.84% examples, 26565 words/s, in_qsize 7, out_qsize 0
2018-03-07 13:55:37,246 : INFO : EPOCH 1 - PROGRESS: at 69.55% examples, 26747 words/s, in_qsize 7, out_qsize 0
2018-03-07 13:55:38,248 : INFO : EPOCH 1 - PROGRESS: at 80.49% examples, 26710 words/s, in_qsize 7, out_qsize 0
2018-03-07 13:55:39,284 : INFO : EPOCH 1 - PROGRESS: at 93.57% examples, 26987 words/s, in_qsize 8, out_qsize 0
2018-03-07 13:55:39,594 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-03-07 13:

Completed pass 2 at alpha 0.000100
Model Saved into Disk
CPU times: user 8h 29min 34s, sys: 2min 41s, total: 8h 32min 15s
Wall time: 2h 31min 34s
